# Datetime advanced

## Dealing with date, time, timedelta in python

* [now](#Now)
* [timezone](#Timezones)
* [Truncate](#Truncate)
* [Timedelta](#Timedelta)
* [(to and from) Strings](#Strings)
* [Unix timestamp](#Unix_timestamp)

In [2]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz
from dateutil.relativedelta import *

# Now

In [3]:
# in local timezone
datetime.now()

datetime.datetime(2018, 11, 14, 15, 26, 32, 61349)

In [5]:
# in utc
datetime.utcnow()

datetime.datetime(2018, 11, 14, 23, 27, 11, 965610)

# Timezones

#### ref.



[document for pytz](http://pytz.sourceforge.net/#localized-times-and-date-arithmetic)

## Define timezone

In [6]:
pst = timezone('US/Pacific-New')
cst = timezone('Asia/Shanghai') # china standard time
utc = pytz.utc

#### Get a list of time zones

In [7]:
# pytz.all_timezones

## "Localize": adding time zone information to datetime
adding info that "this is in utc time"

In [8]:
## adding info that "this is in utc time"
utc_time = utc.localize(datetime(2018, 8, 23, 22, 52, 8))

In [9]:
utc_time

datetime.datetime(2018, 8, 23, 22, 52, 8, tzinfo=<UTC>)

In [10]:
utc_time.tzinfo # this is the timezone information that can be used to "localize" a naive datetime object

<UTC>

This is the timezone information that can be used to "localize" a naive datetime object

## Convert timezones

In [13]:
# the original time should carry a timezone (be 'localized' to a timezone)
pst_time = utc_time.astimezone(pst)

print('UCT: {}'.format(utc_time))
print('PST: {}'.format(pst_time))

UCT: 2018-08-23 22:52:08+00:00
PST: 2018-08-23 15:52:08-07:00


### Timezone-safe comparison

In [11]:
print(utc_time)
print(pst_time)

utc_time == pst_time

2018-08-23 22:52:08.797172+00:00
2018-08-23 15:52:08.797172-07:00


True

# Truncate


## Truncate datetime

In [20]:
time1 = pst.localize(datetime(2018, 8, 23, 20, 52, 8, 0))
truncated_time1 = time1.replace(hour=0, minute=0, second=0, microsecond=0)

print('time1 :  ' + str(time1))
print('Truncated to date :  ' + str(truncated_time1))

time1 :  2018-08-23 20:52:08-07:00
Truncated to date :  2018-08-23 00:00:00-07:00


In [21]:
time2 = pst.localize(datetime(2018, 8, 23, 15, 52, 8, 0))
truncated_time2 = time2.replace(hour=0, minute=0, second=0, microsecond=0)

print('time2 :  ' + str(time2))
print('Truncated to date :  ' + str(truncated_time2))

time2 :  2018-08-23 15:52:08-07:00
Truncated to date :  2018-08-23 00:00:00-07:00


In [22]:
## After truncation should be the same
truncated_time1 == truncated_time2

True

## Truncate timedelta

In [38]:
def truncate_microseconds(delta):
    '''
    https://stackoverflow.com/questions/18470627/python-timedelta-remove-microseconds
    '''
    return delta - timedelta(microseconds=delta.microseconds)

In [39]:
time1 = pst.localize(datetime(2018, 8, 23, 20, 52, 8, 103))
time2 = pst.localize(datetime(2018, 8, 23, 20, 50, 8, 0))

# print('time 1 :  ' + str(time1))
# print('time 2 :  ' + str(time2))

print('Time delta: {}'.format(time1 - time2))
print('Time delta: {}'.format(truncate_microseconds(time1 - time2)))

Time delta: 0:02:00.000103
Time delta: 0:02:00


# Timedelta

* [timedelta to string](#timedelta_to_string)

## Sum timedelta: yes you can

In [41]:
sum([timedelta(1) , timedelta(2, seconds = 2)], timedelta())

datetime.timedelta(3, 2)

In [19]:
print(sum([timedelta(1) , timedelta(2, seconds = 2)], timedelta()))

3 days, 0:00:02


## Calender timedelta (relative time delta)


See discussion on [stackoverflow](https://stackoverflow.com/questions/35066588/is-there-a-simple-way-to-increment-a-datetime-object-one-month-in-python)

In [42]:
from dateutil.relativedelta import *

### Month to date

In [43]:
pst = timezone('US/Pacific-New')
today = pst.localize(datetime.now())
print(today)

2018-11-14 15:54:45.334779-08:00


In [44]:
month_to_date = today + relativedelta(months=-1)
print(month_to_date)

2018-10-14 15:54:45.334779-08:00


### in a week

In [47]:
next_week = today + relativedelta(weeks=1)
print(next_week)

2018-11-21 15:54:45.334779-08:00


### "The second Tuesday next month"

In [45]:
next_meeting = today + relativedelta(months=+1, day=1, weekday=TU(2))
print(next_meeting)

2018-12-11 15:54:45.334779-08:00


# Strings

## Timestamp_to_string

#### Cheat sheet for most seen formats

In [49]:
ts = datetime.now()

In [25]:
ts.strftime("%Y-%b(%B)-%d %H:%M:%S")

'2018-Oct(October)-02 15:41:17'

In [26]:
ts.strftime("%Y-%m-%d (%a / %A) %H:%M:%S")

'2018-10-02 (Tue / Tuesday) 15:41:17'

In [27]:
utc.localize(ts).strftime("%Y-%m-%d %I:%M (%p)")

'2018-10-02 03:41 (PM)'

In [28]:
utc.localize(ts).strftime("%Y-%m-%d %I:%M %p (%Z)")

'2018-10-02 03:41 PM (UTC)'

## String to timestamp

In [51]:
ts_str = utc.localize(ts).astimezone(pst).strftime("%Y-%m-%d %I:%M %p (%z)")

In [52]:
ts_str 

'2018-11-14 07:57 AM (-0800)'

In [53]:
ts2 = datetime.strptime(ts_str,"%Y-%m-%d %I:%M %p (%z)")
ts2

datetime.datetime(2018, 11, 14, 7, 57, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600)))

### Hour (as a number) to string

1. Direct formating
2. `print` timedelta
3. make a trivial datetime and `strftime`

In [54]:
"%02d:00" % 1

'01:00'

In [55]:
h=15.3
print(timedelta(hours = h))

15:18:00


In [56]:
td = timedelta(hours = h)

### timedelta_to_string

In [57]:
def timedelta2str(td):
    '''
    if the str(td) does not work
    '''
    (h, res) = divmod(int(td.total_seconds()), 3600)
    (m, s) = divmod(res, 60)
    return '{:02}:{:02}:{:02}'.format(h,m,s)

In [58]:
timedelta2str(td)

'15:18:00'

In [59]:
(datetime(1,1,1) + timedelta(hours = h)).strftime("%I:%M %p")

'03:18 PM'

# Unix_timestamp

(Bigint, 1494032110 stands for 2017-05-06 08:55:10 CST)

In [33]:
utc.localize(datetime.utcfromtimestamp(1494032110)).astimezone(cst)

datetime.datetime(2017, 5, 6, 8, 55, 10, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>)